# Aufgabe 2 - Hauptproduktionsprogramm

## Mathematisches Modell

**Zielfunktion**

\begin{equation}
	\min\ \ K = \sum^{K}_{k=1} \sum^{J}_{j=1} k_{Lj} \cdot x_{L_{jk}} + \sum^{K}_{k=1} \sum^{I}_{i=1} u_{i} \cdot U_{ik}
\end{equation}

**unter den Nebenbedingungen**

\begin{align}
&&  x_{L_{j,k-1}} +  x_{jk} - r_{jk} &= x_{L_{jk}}  && \forall j, \forall k \\[5pt]
&&  \sum^{J}_{j=1} \sum^{Z_{j}}_{z=0} f_{jiz} \cdot x_{j,k+z} - U_{ik} &\leq b_{ik} && \forall i, \forall k \\[5pt]
&&  U_{ik} &\leq U_{ik,\max} && \forall k \\[5pt]
&& x_{L_{j0}} &= \bar{x}_{L_{j0}} && \\[10pt]

&& x_{jk}, \ x_{L_{jk}}, \ U_{ik} &\geq 0 && \forall j, \forall k, \forall i
\end{align}

## Daten

### Kapazitätsbelastung

| Vorlaufperiode | 2 | 1 | 0 |
|----------------|---|---|---|
| **Produkt A**  |   |   |   |
| Segment 1      |   |   | 1 |
| Segment 2      |   | 4 |   |
| Segment 3      |   |   |   |
| **Produkt B**  |   |   |   |
| Segment 1      |   |   | 2 |
| Segment 2      |   | 3 |   |
| Segment 3      | 3 | 4 |   |

### Nachfrage

| Periode       | 1 | 2  | 3  | 4  | 5  | 6  | 7  | 8  |
|:--------------|---|----|----|----|----|----|----|----|
| **Produkt A** |   | 45 | 30 | 40 | 30 | 50 | 10 | 60 |
| **Produkt B** |   |    | 25 | 30 | 25 | 30 | 20 | 30 |

### Weitere Daten

Die "normalen" Produktionskapazitäten betragen in Produktionssegment 1 100 Einheiten, in Segment 2 150 Einheiten und in Segment 3 120 Einheiten pro Periode. Hinzu kommt eine mögliche Zusatzkapazität von 100 Einheiten in jedem Produktionssegment, wobei mit einer zusätzlich genutzten Einheit in Segment 1 7 GE, in Segment 2 8 GE und in Segment 3 9 GE weitere Kosten anfallen. Für die Lagerung werden bei Produkt A 20 GE benötigt und bei Produkt B 50 GE. Der Anfangslagerbestand beträgt für beide Produkte 0 ME.

## Aufgabe 2b)
Lösen Sie das Modell zur Hauptprogrammplanung, indem Sie den mengenbasierten Syntax verwenden.

In [ ]:
import gurobipy as gp
from gurobipy import GRB

### Definition der Mengen

In [ ]:
K_max = 8
J_max = 2
I_max = 3
Z_max = 3 # Vorlaufperioden: 0, 1, 2

K = range(K_max) # K läuft nun von 0-7 (also um eins versetzt zur Aufgabenstellung)
J = range(J_max)
I = range(I_max)
Z = range(Z_max)

### Dateneingabe
Kosten für Lagerung und Überstunden:

In [ ]:
kL = [20,50]
u = [7,8,9]

Bedarfe:

In [ ]:
r = [[0,45,30,40,30,50,10,60],
	 [0,0,25,30,25,30,20,30]]

Produktionskapazitäten und maximale Überstunden:

In [ ]:
b = [[100,100,100,100,100,100,100,100],
	 [150,150,150,150,150,150,150,150],
	 [120,120,120,120,120,120,120,120]]

Umax = [[100,100,100,100,100,100,100,100],
     [100,100,100,100,100,100,100,100],
	 [100,100,100,100,100,100,100,100]]

Kapazitätsbelastungsfaktoren mit Berücksichtigung der Vorlaufperioden:

In [ ]:
f = [[[1,0,0],[0,4,0],[0,0,0]],
	 [[2,0,0],[0,3,0],[0,4,3]]]
# Hier umgekehrte Reihenfolge als bei der Tabelle (z läuft von 0-2)

### Initialisierung des Modells und der Variablen

In [ ]:
m = gp.Model()

In [ ]:
x = m.addVars(J_max, K_max, vtype=GRB.INTEGER, name="x")
xL = m.addVars(J_max, K_max, vtype=GRB.INTEGER, name="xL")
U = m.addVars(I_max, K_max, vtype=GRB.INTEGER, name="U")

### Definition der Zielfunktion

In [ ]:
m.setObjective(
    gp.quicksum(kL[j] * xL[j,k] for j in J for k in K) 
    + gp.quicksum(u[i] * U[i,k] for i in I for k in K),
    GRB.MINIMIZE)

### Hinzufügen der Nebenbedingungen

In [ ]:
for j in J:
    m.addConstr(xL[j,0] == x[j,0] - r[j][0], name="LAE"+str(j))

In [ ]:
for j in J:
    for k in K: 
        if k > 0:
            m.addConstr(xL[j,k] == xL[j,k-1] + x[j,k] - r[j][k], name="LB"+str(j)+str(k))

In [ ]:
for i in I:
    for k in K:
        if k <= K_max-Z_max:
            m.addConstr(gp.quicksum(f[j][i][z] * x[j,k+z] for j in J for z in Z) - U[i,k] <= b[i][k], name="Kap"+str(i)+str(k))

In [ ]:
for k in K:
    for i in I:
        m.addConstr(U[i,k] <= Umax[i][k], name="Zusatz"+str(k)+str(i))

### Optimierung

In [ ]:
m.optimize()

## Aufgabe 1c)
Im Folgenden sollen die Ergebnisse in der Konsole ausgegeben werden. Schreiben Sie in die Konsole `Ergebnisse der Hauptproduktionsprogrammplanung`.

In [ ]:
print("\nErgebnisse der Hauptproduktionsprogrammplanung \n")

## Aufgabe 1d)
Formulieren Sie einen Satz, welcher die gesamten Kosten angibt.

In [ ]:
print("Die minimalen Kosten betragen", m.getAttr(GRB.Attr.ObjVal),"GE.")

## Aufgabe 1e)
Geben Sie den optimalen Produktionsplan in der Konsole aus.

In [ ]:
for j in J:
        for k in K:
            print("Von Produkt", j, "wird in Periode", k, " ", x[j,k].X, "Einheiten hergestellt.")

## Aufgabe 1f)
Lassen Sie sich den Schlupf der Kapazitätsrestriktion ausgeben! Beschränken Sie sich dabei auf Produktionssegment 1. Versuchen Sie auch den Schattenpreis zu ermitteln.

In [ ]:
for k in K:
    if k <= K_max - Z_max:
        capaConstr = m.getConstrByName("Kap0"+str(k))
        print("In Periode", k, "weist die Kapazität einen Schlupf von", capaConstr.Slack, "ME auf.")

In [ ]:
for k in K:
    if k <= K_max - Z_max:
        capaConstr = m.getConstrByName("Kap0"+str(k))
        print("In Periode", k, "hat die Kapazität einen Schattenpreis von", capaConstr.Pi)
# Wert kann nicht berechnet werden!

## Aufgabe 1g)
Wieviele Überstunden sind insgesamt notwendig?

In [ ]:
overtime = sum(U[i,k].X for i in I for k in K)
print("Insgesamt wurden", overtime, "Überstunden gemacht.")

## Aufgabe 1h)
Wie hoch sind die gesamten Lagerkosten?

In [ ]:
storageCosts = sum(xL[j,k].X * kL[j] for j in J for k in K)
print("Die Lagerkosten betragen insgesamt", storageCosts, "GE.")